In [1]:
import pandas as pd
from tkinter import filedialog, Tk  

file_path = filedialog.askopenfilename(title="pro's kit 작업파일", defaultextension=".xlsx") 
pi = pd.read_excel(file_path, header=0, sheet_name=0)   # PI 서류를 첫번째 시트에 놓기
prokit = pd.read_excel(file_path, header=0, sheet_name=1) #Prokit 재고조사2 서류를 두번째 시트에 놓기

In [2]:
pi = pi.rename(columns={'description' : '상세정보'})
pi = pi.rename(columns={'Item No.' : '주문명'})
pi = pi.rename(columns={'Quantity' : '수량'})
pi_trim = pi[["상세정보", "주문명", "수량"] ]

pi_trim = pi_trim.dropna()

In [3]:
prokit_trim = prokit[['주문코드', '주문명', '제품명']]

merged_df = pd.merge(pi_trim, prokit_trim, on="주문명", how="left")
merged_df = merged_df[['주문코드', '주문명', '제품명', '상세정보', '수량']]
merged_df['수량'] = pd.to_numeric(merged_df['수량'], errors="coerce")
merged_df = merged_df.fillna(value="확인필요")
merged_df['수량'] = merged_df['수량'].astype(int)
merged_df

,주문코드,주문명,제품명,상세정보,수량
0,T9160,19400-M5,"PROKIT (19400-M5), 뽁스 드라이버",Electronic Hex Nut Drivers (5.0 Size),300
1,T8308,1PK-101T,"PROKIT (1PK-101T), 절연 핀셋",Insulated Tweezer (120mm),150
2,T8314,1PK-119T,"PROKIT (1PK-119T), 포인트 핀셋",Point Tweezer (200mm),50
3,T9447,1PK-125T,"PROKIT (1PK-125T), 핀셋",Super Fine Tip Straight Tweezer (120mm),100
4,PK539,1PK-3001E,"PROKIT (1PK-3001E), 스트리퍼(절연 손잡이)",Precision Wire Stripper With Conductive Handle,60
...,...,...,...,...,...
132,확인필요,CP-335N,확인필요,CAT.5/6/7 Modular,50
133,확인필요,MT-7029N,확인필요,Noise-filtering Network PoE,30
134,확인필요,PM-754,확인필요,Flat Nose Plier (135mm),56
135,확인필요,CP-420,확인필요,5 in 1 Long Nose Multi-tool,50


In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from tkinter import filedialog, Tk
from tqdm import tqdm
from requests.exceptions import MissingSchema


login_url = "https://comsmart.co.kr/cmart/bbs/login_check.php"
username = "fred"
password = "123"

payload = {
    'mb_id': username,
    'mb_password': password
}

session = requests.Session()
res = session.post(login_url, data=payload)

file_path = filedialog.askopenfilename(title="제품 리스트파일", defaultextension=".xlsx")
file_list = pd.read_excel(file_path, header=0, sheet_name=0)

final_model_name = []
final_link_address = []
final_description_list = []
final_access_link_list = []
final_stock_list = []

for r in tqdm(range(len(file_list))):
    url = "https://www.comsmart.co.kr/cmart/shop/search.php?wk=&q="
    item_name = str(file_list.loc[r, '모델명'])
    url_link = url + item_name
    comsmart = session.get(url_link)
    soup = BeautifulSoup(comsmart.text, 'html.parser')

    table = soup.select("div.tgslider")

    model_name = []
    image_link = []
    description_list = []
    access_link_list = []


    for i in table:
        for e in i.select("div.red.it_5") :
            model = e.get_text().replace("[","").replace("]","").strip()
            model_name.append(model)

    for i in table:
        for e in i.select("div.inner_inner img") :
            link = e['src']
            image_link.append(link)

    for i in table :
        for e in i.select("div.it_name") :
            description = e.get_text()
            description_list.append(description)

    for i in table :
        for e in i.select("div.inner_inner a") :
            access_link = e['href']
            access_link_list.append(access_link)

    df = pd.DataFrame({"모델명" : model_name, "링크주소" : image_link, "상세정보" : description_list, "접속링크" : access_link_list})


    image_link_address = df[df['모델명']  == item_name]['링크주소']
    if not image_link_address.empty:
        image_link_address = image_link_address.values[0]
    else:
        image_link_address = ""

    description_list_df = df[df['모델명']  == item_name]['상세정보']
    if not description_list_df.empty:
        description_list_df = description_list_df.values[0]
    else:
        description_list_df = ""

    access_link_list_df = df[df['모델명']  == item_name]['접속링크']
    if not access_link_list_df.empty:
        access_link_list_df = access_link_list_df.values[0]
    else:
        access_link_list_df = ""



    try : 
        product_url = access_link_list_df
        stock = session.get(product_url)
        soup = BeautifulSoup(stock.text, 'html.parser')
        access_link_table = soup.select("#goods_price_inner tr")

        for i in access_link_table :
            for e in i.select("#erpchk") :
                stocks = e.get_text().split()
                if stocks[0] == "총재고" :
                    total_stocks = stocks[2]
                    
    except MissingSchema:
        total_stocks = ""


    final_model_name.append(item_name)
    final_link_address.append(image_link_address)
    final_description_list.append(description_list_df)
    final_stock_list.append(total_stocks)

final_df = pd.DataFrame( {"모델명" : final_model_name, "상세정보" : final_description_list, "총재고" : final_stock_list, "이미지주소" : final_link_address})


from IPython.display import display, HTML

# 각 이미지 링크를 HTML 이미지 태그로 변환
final_df['이미지주소'] = final_df['이미지주소'].apply(lambda x: f'<img src="{x}" width="100">')
# 데이터프레임을 HTML로 변환
html = final_df.to_html(escape=False)

with open('output.html', 'w') as f:
    f.write(html)

100%|██████████| 86/86 [00:55<00:00,  1.56it/s]
